## Task1

In [0]:
def skew(seq):
  c = g = 0
  ls = [0]
  for ch in seq:
    if ch == 'C':
      c += 1
    if ch == 'G':
      g += 1
    ls.append((g-c))
  mn = min(ls)
  return [idx for idx, val in enumerate(ls) if val == mn]
    

In [7]:
print(*skew('CCTATCGGTGGATTAGCATGTCCCTGTACGTTTCGCCGCGAACTAGTTCACACGGCTTGATGGCAAATGGTTTTTCCGGCGACCGTAATCGTCCACCGAG'))

53 97


## Task2

In [0]:
from collections import defaultdict

In [0]:
def clump_find(seq, k, L, t):
  mp = defaultdict(int)
  ans = dict()
  
  for idx in range(L-k+1):
    mp[seq[idx:idx+k]] += 1
    if mp[seq[idx:idx+k]] >= t:
      #print(seq[idx:idx+k])
      ans[seq[idx:idx+k]] = True
    
  start = 0
  end = L - k
  
  while end < len(seq) - k:
    mp[seq[idx:idx+k]] -= 1
    start += 1
    end += 1
    mp[seq[end:end+k]] += 1
    if mp[seq[end:end+k]] >= t:
      #print(seq[end:end+k])
      ans[seq[end:end+k]] = True

  return [key for key in ans]
    
  

In [23]:
print(*clump_find('CGGACTCGACAGATGTGAAGAAATGTGAAGACTGAGTGAAGAGAAGAGGAAACACGACACGACATTGCGACATAATGTACGAATGTAATGTGCCTATGGC', 5, 75, 4))

GAAGA CGACA AATGT


## Task3

In [0]:
from itertools import product

In [0]:
def hamming_distance(p, q):
  return sum([q[idx] != ch for idx,ch in enumerate(p)])

In [0]:
def calculate_score(k, k_mer, dna):
  score = 0
  for d in dna:
    all_substring = [d[idx:idx+k] for idx in range(len(d)-k+1)]
    score += min(map(lambda x: hamming_distance(x,k_mer), all_substring))
  return (score, k_mer)

In [0]:
def median_string_find():
  dna = []
  with open('input.txt','r') as file:
    for line in file:
      dna.append(line.strip())
  k = int(dna.pop(0))
  all_k_mer = [''.join(x) for x in product(['A','C','G','T'],repeat = k)]
  ans = ''
  score = 1000000
  for k_mer in all_k_mer:
    x, y = calculate_score(k, k_mer, dna)
    #print(x,y)
    if score > x:
      ans = y
      score = x
  return ans  

In [52]:
print(median_string_find())

ACG


## Task4

In [0]:
from functools import reduce

In [0]:
def profile_most_probable_k_mer():
  seq = input()
  k = int(input())
  profile_matrix = []
  mp = {
      'A' : 0,
      'C' : 1,
      'G' : 2,
      'T' : 3
  }
  for _ in range(4):
    profile_matrix.append(list(map(float, input().split())))
  ans = ''
  score = 0
  for idx in range(len(seq)-k+1):
    s = seq[idx:idx+k]
    temp_score = 1.0
    
    for idx, ch in enumerate(s):
      pos = mp[ch]
      temp_score *= profile_matrix[pos][idx]
    if temp_score > score:
      score = temp_score
      ans = s
  return ans
      

    

In [71]:
profile_most_probable_k_mer()

ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT
5
0.2 0.2 0.3 0.2 0.3
0.4 0.3 0.1 0.5 0.1
0.3 0.3 0.5 0.2 0.4
0.1 0.2 0.1 0.1 0.2


'CCGAG'

## Task5

In [0]:
k = 3
t = 5
dna = [
    'GGCGTTCAGGCA',
    'AAGAATCAGTCA',
    'CAAGGAGTTCGC',
    'CACGTCAATCAC',
    'CAATAATATTCG'
]

In [0]:
def get_most_probable_k_mer(k, seq, profile_matrix):
  mp = {
      'A' : 0,
      'C' : 1,
      'G' : 2,
      'T' : 3
  }
  #print(profile_matrix)
  ans = ''
  score = 0
  for idx in range(len(seq)-k+1):
    s = seq[idx:idx+k]
    temp_score = 1.0
    
    for idx, ch in enumerate(s):
      pos = mp[ch]
      temp_score = temp_score * profile_matrix[pos][idx]
    if temp_score > score:
      score = temp_score
      ans = s
  return ans
  
  
  

In [0]:
def get_profile_matrix(k, motif_set):
  profile = [[],[],[],[]]
  for i in range(k):
    val = (sum(list(map(lambda x: x[i] == 'A', motif_set))) + 1)/(4 + len(motif_set))
    profile[0].append(val)
    val = (sum(list(map(lambda x: x[i] == 'C', motif_set))) + 1)/(4 + len(motif_set))
    profile[1].append(val)
    val = (sum(list(map(lambda x: x[i] == 'G', motif_set))) + 1)/(4 + len(motif_set))
    profile[2].append(val)
    val = (sum(list(map(lambda x: x[i] == 'T', motif_set))) + 1)/(4 + len(motif_set))
    profile[3].append(val)
  #endfor
  return profile
  
  

In [0]:
def find_best_k_mer_and_add_in_temp_motif_set(k, seq, motif_set):
  profile_matrix = get_profile_matrix(k, motif_set)
  motif_set.append(get_most_probable_k_mer(k, seq,profile_matrix))
  

In [0]:
from collections import defaultdict
import copy

In [0]:
def calculate_motif_set_score(t, k, motif_set):
  score = 0
  for idx in range(k):
    mp = defaultdict(int)
    for s in motif_set:
      mp[s[idx]] += 1
    score += (t - max([val for key,val in mp.items()]))
  return score

In [0]:
def greedy_motif_search():
  current_motif_set = [s[0:k] for s in dna]
  for idx in range(len(dna[0])-k+1):
    k_mer = dna[0][idx:idx+k]
    temp_motif_set = [k_mer]
    for i in range(1,t):
      find_best_k_mer_and_add_in_temp_motif_set(k, dna[i], temp_motif_set)
    if calculate_motif_set_score(t,k,current_motif_set) > calculate_motif_set_score(t,k,temp_motif_set):
      current_motif_set = copy.copy(temp_motif_set)
    #endif
    #endfor
  #endfor
  return current_motif_set
    
  

In [146]:
ls = greedy_motif_search()
for l in ls:
  print(l)

TTC
ATC
TTC
ATC
TTC
